In [31]:
import os
import re
import json
import tqdm
import traceback

def get_contents(input_file_path):
    pattern_pragma = r'pragma\s+solidity\s+(\^?>?=?\d+\.\d+\.\d+);'
    pattern_import = r"import\s+[\'\"](.+?)[\'\"]\;"
    pattern_from = r"import\s*{.*}\s*from\s*[\'\"](.+?)[\'\"]\;"
    
    with open(input_file_path, 'r') as f:
        data = f.read()
    data = re.sub(pattern_import, '', data)
    data = re.sub(pattern_pragma, '', data)
    data = re.sub(pattern_from, '', data)
    data = delete_comments(data)
    data = "\n".join([i for i in data.split("\n") if i.strip() != ""])
    
    # tmppath = os.path.join(os.path.dirname(input_file_path), "tmp.sol")
    # with open(tmppath, 'w') as f:
    #     f.write(data)
    
    # with open(output_file_path, 'a') as output_file:
    #     output_file.write(data)
    #     output_file.write('\n')
    # os.remove(tmppath)
    
    data += '\n'
    return data

def delete_comments(allstr):
    allstrs = allstr.split("\n")
    mark = 1
    newstr = ""
    for str in allstrs:
        strs = str.split("\"")
        for i in range(len(strs)):
            if mark == 0:
                if strs[i].find("*/") !=-1:
                    ss = strs[i].split("*/")
                    if len(ss) >= 1:
                        newstr += ss[1]
                    mark = 1
                    continue
                else:
                    continue
            if i % 2 == 0 and mark == 1:
                if strs[i].find("//") !=-1:
                    ss = strs[i].split("//")
                    newstr += ss[0]
                    break
                if strs[i].find("/*") !=-1:
                    ss = strs[i].split("/*")
                    newstr += ss[0]
                    if strs[i].find("*/") !=-1:
                        ss2 = ss[1].split("*/")
                        newstr += ss2[1]
                    else:
                        mark = 0
                    continue
            newstr += strs[i]
            if i != len(strs)-1 : newstr += "\""
        newstr += "\n"
    return newstr

def file_import(filepath, G:dict = None, original_dict:dict = None, delete_dict:dict = None):
    if G is None:
        G = {}
    if original_dict is None:
        original_dict = {}
    if delete_dict is None:
        delete_dict = {}
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = f.read()
        # pattern = r"(import\s+[\'\"](.+?)[\'\"]\;)|(import\s+[\'\"](.+?)[\'\"]\s*from\s*[\'\"](.+?)[\'\"]\;)"
        pattern_import = r"import\s+[\'\"](.+?)[\'\"]\;"
        pattern_from = r"import\s*{.*}\s*from\s*[\'\"](.+?)[\'\"]\;"
        
        file_name = os.path.basename(filepath)
        # print(file_name)
        if file_name not in G:
            G[file_name] = []
        if file_name not in original_dict:
            matches = re.findall(pattern_import + '|' + pattern_from, data)
            need_add_import = []
            for match in matches:
                if match[0]:
                    matchname = match[0]
                else:
                    matchname = match[1]
                # print(matchname)
                # print(matchname[2:])
                matchpath = os.path.join(os.path.dirname(filepath), matchname[2:])
                # print(matchpath)
                if os.path.exists(matchpath):
                    file_import(matchpath, G, original_dict, delete_dict)
                if matchname not in G[file_name]:
                    G[file_name].append(matchname)
                if match[1] and matchname not in need_add_import:
                    need_add_import.append(matchname[2:])

                
            original_dict[file_name] = get_contents(filepath)

            delete_dict[file_name] = get_contents(filepath)
            # print(file_name)
            # print(delete_dict[file_name])
            
            # print(need_add_import)
                
            # tmp = delete_dict[file_name] + '\n'
            if need_add_import:
                # print(1)
                tmp:str = ""
                # delete_dict[file_name] = None
                # print(delete_dict[file_name])
                for name in need_add_import:
                    # print(name)
                    # print(delete_dict[name])
                    # print(type(delete_dict[name]))
                    # tmp = tmp + '\n'
                    tmp = tmp + delete_dict[name]
                    # print(type(tmp))
                    del delete_dict[name]
                delete_dict[file_name] = tmp + delete_dict[file_name]
                # print(delete_dict[file_name])
                    
        
    except Exception as e:
        pass
    
    return G, original_dict, delete_dict

filepath = "/home/lxm/solidity/a/01_12_GuaranteeFund.sol"
G, original_dict, delete_dict = file_import(filepath)
# print(get_contents(filepath))
for k, v in delete_dict.items():
    print(k)
    print(v)

02_12_Ownable.sol
abstract contract Ownable is Context {
    address private _owner;
    event OwnershipTransferred(address indexed previousOwner, address indexed newOwner);
    constructor() {
        _transferOwnership(_msgSender());
    }
    modifier onlyOwner() {
        _checkOwner();
        _;
    }
    function owner() public view virtual returns (address) {
        return _owner;
    }
    function _checkOwner() internal view virtual {
        require(owner() == _msgSender(), "Ownable: caller is not the owner");
    }
    function renounceOwnership() public virtual onlyOwner {
        _transferOwnership(address(0));
    }
    function transferOwnership(address newOwner) public virtual onlyOwner {
        require(newOwner != address(0), "Ownable: new owner is the zero address");
        _transferOwnership(newOwner);
    }
    function _transferOwnership(address newOwner) internal virtual {
        address oldOwner = _owner;
        _owner = newOwner;
        emit OwnershipTran

In [33]:
def get_version(input_file, output_file):
    pattern = r'pragma\s+solidity\s+(\^?>?=?\d+\.\d+\.\d+);'
    with open(input_file, 'r') as f:
        content = f.read()

    matches = re.findall(pattern, content)
    with open(output_file, 'w') as matched_f:
        last_end = 0
        for match in matches:
            start_index = match.start()
            # end_index = match.end()
            matched_content = match.group(0)
            preceding_content = content[last_end:start_index]
            matched_f.write(preceding_content + matched_content + '\n')
            # last_end = end_index
        # remaining_content = content[last_end:]
        # print(remaining_content)
        # remaining_f.write(remaining_content)

filepath = '/home/lxm/solidity/a/0x5f73dbb6f3fa6444b11a8e1880dce25c5e7ad862_newNFT/newNFT.sol'
get_version(filepath, './A_version.sol')